In [1]:
!pip install python-chi python-blazarclient python-dotenv

In [2]:
# if running on local system (not chameleon jupyter)
from dotenv import load_dotenv
import chi
load_dotenv(".env")


True

In [3]:
from chi import context
context.check_credentials()

Username: mr7375@nyu.edu
Currently site: CHI@TACC
Currently project: CH-23118
Projects:
	 CHI-231138
Authentication is valid.


True

In [4]:
chi.set('project_name', 'CH-23118')
# Select your site
chi.use_site('CHI@TACC')

Now using CHI@TACC:
URL: https://chi.tacc.chameleoncloud.org
Location: Austin, Texas, USA
Support contact: help@chameleoncloud.org


In [6]:
from chi import lease
import datetime

l = lease.Lease(
    name="mlsys_gigaio_gpu",
    duration=datetime.timedelta(hours=6),
)

# Correct for GigaIO GPU pool
l.add_node_reservation(
    amount=1,
    node_type="compute_liqid"
)

l.submit(idempotent=True)
l.add_fip_reservation(amount=1)


Waiting for lease to start...


The python binding code in neutronclient is deprecated in favor of OpenstackSDK, please use that as this will be removed in a future release.


Lease mlsys_gigaio_gpu has reached status active


In [7]:
from chi import server
s = server.Server('mlsys-bloat-mr7375',
                  reservation_id=l.node_reservations[0]["id"],
                  image_name="CC-Ubuntu24.04-CUDA",
                  flavor_name="baremetal",
                  key_name="sparkle",
                  network_name="sharednet1"
                  
)
s.submit(idempotent=True)


The python binding code in neutronclient is deprecated in favor of OpenstackSDK, please use that as this will be removed in a future release.


Waiting for server mlsys-bloat-mr7375's status to become ACTIVE. This typically takes 10 minutes for baremetal, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,mlsys-bloat-mr7375
Id,09b9069c-191c-4a95-abe0-b32b700df47f
Status,ACTIVE
Image Name,CC-Ubuntu24.04-CUDA
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.1.82 (v4) Type: fixed MAC: 34:80:0d:ed:52:26
Network Name,sharednet1
Created At,2026-01-21T21:18:56Z
Keypair,sparkle
Reservation Id,14fd71d8-214f-4fa4-8811-ccdd661aac52
Host Id,7c346a87f6e9c6f9d67fb076fe5d76102c3a39bab2eac416bf31a8b7


In [10]:
s.associate_floating_ip()
s.check_connectivity()

The python binding code in neutronclient is deprecated in favor of OpenstackSDK, please use that as this will be removed in a future release.
The python binding code in neutronclient is deprecated in favor of OpenstackSDK, please use that as this will be removed in a future release.


Checking connectivity to 129.114.109.0 port 22.


Connection successful


In [11]:
import os
connect_kwargs = {"key_filename": os.getenv("KEY_PATH")}

In [12]:
s.execute("sudo apt update", connect_kwargs=connect_kwargs)


/Users/megh/nyu/research/mlsys-debloat/.venv/lib/python3.14/site-packages/paramiko/client.py:885: UserWarning: Unknown ssh-ed25519 host key for 129.114.109.0: b'11b926b1257ae6b98ba21bc52f8ee6d4'
  warnings.warn(




Get:1 http://security.ubuntu.com/ubuntu noble-security InRelease [126 kB]
Hit:2 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64  InRelease [1581 B]
Get:5 http://security.ubuntu.com/ubuntu noble-security/main amd64 Packages [1408 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64  Packages [1159 kB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease [126 kB]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates/main amd64 Packages [1697 kB]
Get:9 http://security.ubuntu.com/ubuntu noble-security/main amd64 Components [21.5 kB]
Get:10 http://security.ubuntu.com/ubuntu noble-security/main amd64 c-n-f Metadata [9772 B]
Get:11 http://security.ubuntu.com/ubuntu noble-security/universe amd64 Packages [924 kB]
Get:12 http://security.ubuntu.com/ubuntu

<Result cmd='sudo apt update' exited=0>

In [13]:
s.execute("curl -sSL https://get.docker.com/ | sudo sh", connect_kwargs=connect_kwargs)
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER", connect_kwargs=connect_kwargs)
     

s.execute("curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg \
  && curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list | \
    sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' | \
    sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list", connect_kwargs=connect_kwargs)
s.execute("sudo apt update", connect_kwargs=connect_kwargs)
s.execute("sudo apt-get install -y nvidia-container-toolkit", connect_kwargs=connect_kwargs)
s.execute("sudo nvidia-ctk runtime configure --runtime=docker", connect_kwargs=connect_kwargs)
# for https://github.com/NVIDIA/nvidia-container-toolkit/issues/48
s.execute("sudo jq 'if has(\"exec-opts\", connect_kwargs=connect_kwargs) then . else . + {\"exec-opts\": [\"native.cgroupdriver=cgroupfs\"]} end' /etc/docker/daemon.json | sudo tee /etc/docker/daemon.json.tmp > /dev/null && sudo mv /etc/docker/daemon.json.tmp /etc/docker/daemon.json", connect_kwargs=connect_kwargs)
s.execute("sudo systemctl restart docker", connect_kwargs=connect_kwargs)


s.execute("sudo apt update", connect_kwargs=connect_kwargs)
s.execute("sudo apt -y install nvtop", connect_kwargs=connect_kwargs)

# Executing docker install script, commit: f381ee68b32e515bb4dc034b339266aff1fbc460


+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install ca-certificates curl >/dev/null
+ sh -c install -m 0755 -d /etc/apt/keyrings
+ sh -c curl -fsSL "https://download.docker.com/linux/ubuntu/gpg" -o /etc/apt/keyrings/docker.asc
+ sh -c chmod a+r /etc/apt/keyrings/docker.asc
+ sh -c echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu noble stable" > /etc/apt/sources.list.d/docker.list
+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install docker-ce docker-ce-cli containerd.io docker-compose-plugin docker-ce-rootless-extras docker-buildx-plugin docker-model-plugin >/dev/null

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on th

  UNIT                                                                                                                                                     LOAD   ACTIVE SUB       DESCRIPTION
  proc-sys-fs-binfmt_misc.automount                                                                                                                        loaded active running   Arbitrary Executable File Formats File System Automount Point
  sys-bus-pci-drivers-nvidia.device                                                                                                                        loaded active plugged   /sys/bus/pci/drivers/nvidia
  sys-devices-pci0000:00-0000:00:01.1-0000:01:00.0-host0-target0:0:0-0:0:0:0-block-sda-sda1.device                                                         loaded active plugged   MZ7LH480HBHQ0D3 MKFS_ESP
  sys-devices-pci0000:00-0000:00:01.1-0000:01:00.0-host0-target0:0:0-0:0:0:0-block-sda-sda2.device                                                         lo

INFO: Docker daemon enabled and started



                                                                                                                   loaded active running   D-Bus System Message Bus Socket
  dm-event.socket                                                                                                                                          loaded active listening Device-mapper event daemon FIFOs
  docker.socket                                                                                                                                            loaded active running   Docker Socket for the API
  iscsid.socket                                                                                                                                            loaded active listening Open-iSCSI iscsid Socket
  lvm2-lvmpolld.socket                                                                                                                                     loaded active listening LVM2 poll daemon socket
  lxd

...

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  nvtop
0 upgraded, 1 newly installed, 0 to remove and 181 not upgraded.
Need to get 62.8 kB of archives.
After this operation, 180 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu noble/multiverse amd64 nvtop amd64 3.0.2-1 [62.8 kB]


debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 62.8 kB in 0s (127 kB/s)
Selecting previously unselected package nvtop.
(Reading database ... 113904 files and directories currently installed.)
Preparing to unpack .../nvtop_3.0.2-1_amd64.deb ...
Unpacking nvtop (3.0.2-1) ...
Setting up nvtop (3.0.2-1) ...
Processing triggers for man-db (2.12.0-4build2) ...


debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.


<Result cmd='sudo apt -y install nvtop' exited=0>

In [14]:
s.execute("git clone --recurse-submodules https://github.com/meghrathod/mlsys-debloat", connect_kwargs=connect_kwargs)

Cloning into 'mlsys-debloat'...
Submodule 'negativa-ml' (https://github.com/meghrathod/negativa-ml.git) registered for path 'negativa-ml'
Cloning into '/home/cc/mlsys-debloat/negativa-ml'...


Submodule path 'negativa-ml': checked out 'c4dd654fc84cf9d660cdea880062b1547a856676'


<Result cmd='git clone --recurse-submodules https://github.com/meghrathod/mlsys-debloat' exited=0>